In [11]:
import pandas as pd
from janus.utils.make_dataframe_multivariate import create_per_user_dataframe_multivariate

df = pd.read_csv('tests/dataset_summary.csv')
for col in ['revenue', 'cost']:
    df[col] = df[col].astype('float64')

df

,variant_id,exposure_period,exposures,conversions,revenue,cost
0,A,'2022-01-01',100,10,100.0,20.0
1,A,'2022-01-02',200,20,200.0,40.0
2,A,'2022-01-03',100,12,120.0,24.0
3,B,'2022-01-01',100,12,120.0,24.0
4,B,'2022-01-02',110,15,180.0,36.0
5,B,'2022-01-03',200,30,350.0,70.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   variant_id       6 non-null      object 
 1   exposure_period  6 non-null      object 
 2   exposures        6 non-null      int64  
 3   conversions      6 non-null      int64  
 4   revenue          6 non-null      float64
 5   cost             6 non-null      float64
dtypes: float64(2), int64(2), object(2)
memory usage: 416.0+ bytes


In [13]:
conversion_value_cols=['revenue', 'cost']
df2 = create_per_user_dataframe_multivariate(df, conversion_value_cols)

df2

,variant_id,user_id,exposure_period,converted,revenue,cost
0,A,1,2022-01-01,1,10.0,2.0
1,A,2,2022-01-01,1,10.0,2.0
2,A,3,2022-01-01,1,10.0,2.0
3,A,4,2022-01-01,1,10.0,2.0
4,A,5,2022-01-01,1,10.0,2.0
...,...,...,...,...,...,...
165,B,806,2022-01-03,0,0.0,0.0
166,B,807,2022-01-03,0,0.0,0.0
167,B,808,2022-01-03,0,0.0,0.0
168,B,809,2022-01-03,0,0.0,0.0


In [14]:
agg_dict = {
        'user_id': 'count',
        'converted': 'sum',
    }
agg_dict.update({c: 'sum' for c in conversion_value_cols})


df3 = (
    df2
    .groupby(['variant_id', 'exposure_period'])
    .agg(agg_dict)
    .reset_index()
    .rename(columns={
        'user_id': 'exposures',
        'converted': 'conversions',
    })
)
for col in conversion_value_cols:
    df3[col] = df3[col].astype('float64')
df3['conversions'] = df3['conversions'].astype("int64")

df3

,variant_id,exposure_period,exposures,conversions,revenue,cost
0,A,2022-01-01,100,10,100.0,20.0
1,A,2022-01-02,200,20,200.0,40.0
2,A,2022-01-03,100,12,120.0,24.0
3,B,2022-01-01,100,12,120.0,24.0
4,B,2022-01-02,110,15,180.0,36.0
5,B,2022-01-03,200,30,350.0,70.0


In [15]:
df

,variant_id,exposure_period,exposures,conversions,revenue,cost
0,A,2022-01-01,100,10,100.0,20.0
1,A,2022-01-02,200,20,200.0,40.0
2,A,2022-01-03,100,12,120.0,24.0
3,B,2022-01-01,100,12,120.0,24.0
4,B,2022-01-02,110,15,180.0,36.0
5,B,2022-01-03,200,30,350.0,70.0


In [16]:
(df['conversions'] - df3['conversions']).values

array([0, 0, 0, 0, 0, 0])

In [19]:
cols = [
    'variant_id', 
    'exposure_period', 
    'exposures',
    'conversions'
]
pd.DataFrame.compare(df[cols], df3[cols])

Empty DataFrame
Columns: []
Index: []

In [20]:
df.equals(df3)

True